In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append("../..")

In [10]:
import pipit as pp
import pandas as pd
import numpy as np

In [11]:
trace = pp.Trace.from_otf2("/home/rakrish/Downloads/tortuga-otf2-16/")

In [12]:
trace.events

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
0,0.000000e+00,Instant,ProgramBegin,0,6,"{'attributes': {'Attribute 2': 35468}, 'progra..."
1,4.868378e+01,Instant,ProgramBegin,0,7,"{'attributes': {'Attribute 2': 35471}, 'progra..."
2,2.539193e+02,Instant,ProgramBegin,0,5,"{'attributes': {'Attribute 2': 35465}, 'progra..."
3,3.035577e+02,Instant,ProgramBegin,0,4,"{'attributes': {'Attribute 2': 35462}, 'progra..."
4,4.946367e+04,Instant,ProgramBegin,0,3,"{'attributes': {'Attribute 2': 35460}, 'progra..."
...,...,...,...,...,...,...
95611,1.748975e+09,Instant,MeasurementOnOff,0,4,{'measurement_mode': 'MeasurementMode.OFF'}
95612,1.749001e+09,Leave,run,0,12,None
95613,1.749008e+09,Instant,MeasurementOnOff,0,12,{'measurement_mode': 'MeasurementMode.OFF'}
95614,1.749750e+09,Leave,run,0,0,None


In [17]:
trace.numeric_cols

['Timestamp (ns)']

In [6]:
import polars as pl

In [12]:
def pd_to_polars(df):
    """
    Convert a Pandas DataFrame to Polars DataFrame and handle columns
    with int and float categorical dtypes.
    """
    df = df.copy()
    for col in df.columns:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            if pd.api.types.is_integer_dtype(df[col].cat.categories.dtype):
                df[col] = df[col].astype(int)
                print(f"Column [{col}] cast to int")
            elif pd.api.types.is_float_dtype(df[col].cat.categories.dtype):
                df[col] = df[col].astype(float)
                print(f"Column [{col}] cast to float")

    return pl.from_pandas(df)

In [31]:
pdf = pd_to_polars(trace.events[["Timestamp (ns)", "Event Type", "Name", "Thread", "Process"]])
pdf = pdf.with_row_index("index")

Column [Thread] cast to int
Column [Process] cast to int


In [32]:
pdf

index,Timestamp (ns),Event Type,Name,Thread,Process
u32,f64,cat,cat,i64,i64
0,0.0,"""Instant""","""ProgramBegin""",0,6
1,48.683779,"""Instant""","""ProgramBegin""",0,7
2,253.919318,"""Instant""","""ProgramBegin""",0,5
3,303.557681,"""Instant""","""ProgramBegin""",0,4
4,49463.674098,"""Instant""","""ProgramBegin""",0,3
…,…,…,…,…,…
95611,1.7490e9,"""Instant""","""MeasurementOnO…",0,4
95612,1.7490e9,"""Leave""","""run""",0,12
95613,1.7490e9,"""Instant""","""MeasurementOnO…",0,12


In [40]:
%%timeit
trace.events.iloc[200]

56.7 µs ± 356 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [39]:
%%timeit
pdf[200]

1.37 µs ± 34.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [41]:
%%timeit
trace.events[trace.events["Process"] == 0]

499 µs ± 39.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
%%timeit
pdf.filter(pl.col("Process") == 0)

104 µs ± 3.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [34]:
%%timeit
pdf.filter(pl.col("index") == 200)

61.5 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [35]:
%%timeit
pdf.row( by_predicate=(pl.col("index") == 200) )

65.7 µs ± 693 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [38]:
%%timeit
pdf[200]

1.35 µs ± 4.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [46]:
pdf.filter(pl.col("Event Type").is_in(["Enter", "Leave"]))

index,Timestamp (ns),Event Type,Name,Thread,Process
u32,f64,cat,cat,i64,i64
8,59333.11707,"""Enter""","""MPI_Init""",0,5
9,59384.664601,"""Enter""","""MPI_Init""",0,7
10,59404.710863,"""Enter""","""MPI_Init""",0,4
11,59503.987589,"""Enter""","""MPI_Init""",0,6
12,89863.574038,"""Enter""","""MPI_Init""",0,2
…,…,…,…,…,…
95606,1.7489e9,"""Leave""","""run""",0,3
95608,1.7489e9,"""Leave""","""run""",0,9
95610,1.7490e9,"""Leave""","""run""",0,4


In [47]:
import polars as pl
import numpy as np

def new_match_events(df):
    matching_events = np.full(df.height(), np.nan)
    matching_times = np.full(df.height(), np.nan)

    # Only pairing enter and leave rows
    enter_leave_df = df.filter(pl.col("Event Type").is_in(["Enter", "Leave"]))

    # List of processes and/or threads to iterate over
    if "Thread" in df.columns:
        exec_locations = set(zip(df["Process"].to_list(), df["Thread"].to_list()))
        has_thread = True
    else:
        exec_locations = set(df["Process"].to_list())
        has_thread = False

    for curr_loc in exec_locations:
        # Only filter by thread if the trace has a thread column
        if has_thread:
            curr_process, curr_thread = curr_loc
            filtered_df = enter_leave_df.filter(
                (df["Process"] == curr_process) & (df["Thread"] == curr_thread)
            )
        else:
            filtered_df = enter_leave_df.filter(df["Process"] == curr_loc)

        stack = []

        # Iterate through all events of filtered DataFrame
        for row in filtered_df.iterrows():
            curr_df_index = row.get_column("row").get(0)
            curr_timestamp = row.get_column("Timestamp (ns)").get(0)
            evt_type = row.get_column("Event Type").get(0)
            curr_name = row.get_column("Name").get(0)

            if evt_type == "Enter":
                # Add current dataframe index and timestamp to stack
                stack.append((curr_df_index, curr_timestamp, curr_name))
            else:
                # We want to iterate through the stack in reverse order
                # until we find the corresponding "Enter" Event
                enter_name = None
                i = len(stack) - 1
                while enter_name != curr_name and i > -1:
                    enter_df_index, enter_timestamp, enter_name = stack[i]
                    i -= 1

                if enter_name == curr_name:
                    # Remove matched event from the stack
                    del stack[i + 1]

                    # Fill in the lists with the matching values if event found
                    matching_events[enter_df_index] = curr_df_index
                    matching_events[curr_df_index] = enter_df_index

                    matching_times[enter_df_index] = curr_timestamp
                    matching_times[curr_df_index] = enter_timestamp
                else:
                    continue

    df = df.with_column("_matching_event", pl.Series.new("Int32", matching_events))
    df = df.with_column("_matching_timestamp", pl.Series.new("Int32", matching_times))
    return df

In [14]:
%%timeit
trace._match_events()

92.8 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
trace._match_caller_callee()

The slowest run took 20.56 times longer than the fastest. This could mean that an intermediate result is being cached.
3.06 µs ± 4.39 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
df

NameError: name 'df' is not defined

In [51]:
def pl_match_events(df):
    matching_events = np.full(df.height, np.nan)
    matching_times = np.full(df.height, np.nan)

    # Only pairing enter and leave rows
    enter_leave_df = df.filter(pl.col("Event Type").is_in(["Enter", "Leave"]))

    # List of processes and/or threads to iterate over
    if "Thread" in df.columns:
        exec_locations = set(zip(df["Process"].to_list(), df["Thread"].to_list()))
        has_thread = True
    else:
        exec_locations = set(df["Process"].to_list())
        has_thread = False

    for curr_loc in exec_locations:
        # Only filter by thread if the trace has a thread column
        if has_thread:
            curr_process, curr_thread = curr_loc
            filtered_df = enter_leave_df.filter(
                (df["Process"] == curr_process) & (df["Thread"] == curr_thread)
            )
        else:
            filtered_df = enter_leave_df.filter(df["Process"] == curr_loc)

        stack = []

        # Iterate through all events of filtered DataFrame
        for row in filtered_df.iterrows():
            curr_df_index = row.get_column("row").get(0)
            curr_timestamp = row.get_column("Timestamp (ns)").get(0)
            evt_type = row.get_column("Event Type").get(0)
            curr_name = row.get_column("Name").get(0)

            if evt_type == "Enter":
                # Add current dataframe index and timestamp to stack
                stack.append((curr_df_index, curr_timestamp, curr_name))
            else:
                # We want to iterate through the stack in reverse order
                # until we find the corresponding "Enter" Event
                enter_name = None
                i = len(stack) - 1
                while enter_name != curr_name and i > -1:
                    enter_df_index, enter_timestamp, enter_name = stack[i]
                    i -= 1

                if enter_name == curr_name:
                    # Remove matched event from the stack
                    del stack[i + 1]

                    # Fill in the lists with the matching values if event found
                    matching_events[enter_df_index] = curr_df_index
                    matching_events[curr_df_index] = enter_df_index

                    matching_times[enter_df_index] = curr_timestamp
                    matching_times[curr_df_index] = enter_timestamp
                else:
                    continue

    df = df.with_column("_matching_event", pl.Series.new("Int32", matching_events))
    df = df.with_column("_matching_timestamp", pl.Series.new("Int32", matching_times))
    return df

In [52]:
pl_match_events(pdf)

ShapeError: filter's length: 95616 differs from that of the series: 64832

In [40]:
def original_match_events(df):
    matching_events = [float("nan")] * len(df)
    matching_times = [float("nan")] * len(df)
    
    # only pairing enter and leave rows
    enter_leave_df = df.loc[
        df["Event Type"].isin(["Enter", "Leave"])
    ]
    
    # list of processes and/or threads to iterate over
    if "Thread" in df.columns:
        exec_locations = set(zip(df["Process"], df["Thread"]))
        has_thread = True
    else:
        exec_locations = set(df["Process"])
        has_thread = False
    
    for curr_loc in exec_locations:
        # only filter by thread if the trace has a thread column
        if has_thread:
            curr_process, curr_thread = curr_loc
            filtered_df = enter_leave_df.loc[
                (enter_leave_df["Process"] == curr_process)
                & (enter_leave_df["Thread"] == curr_thread)
            ]
        else:
            filtered_df = enter_leave_df.loc[
                (enter_leave_df["Process"] == curr_loc)
            ]
    
        stack = []
    
        # Note: The reason that we are creating lists that are
        # copies of the dataframe columns below and iterating over
        # those instead of using pandas iterrows is due to an
        # observed improvement in performance when using lists.
    
        event_types = list(filtered_df["Event Type"])
        df_indices, timestamps, names = (
            list(filtered_df.index),
            list(filtered_df["Timestamp (ns)"]),
            list(filtered_df.Name),
        )
    
        # Iterate through all events of filtered DataFrame
        for i in range(len(filtered_df)):
            curr_df_index, curr_timestamp, evt_type, curr_name = (
                df_indices[i],
                timestamps[i],
                event_types[i],
                names[i],
            )
    
            if evt_type == "Enter":
                # Add current dataframe index and timestamp to stack
                stack.append((curr_df_index, curr_timestamp, curr_name))
            else:
                # we want to iterate through the stack in reverse order
                # until we find the corresponding "Enter" Event
                enter_name, i = None, len(stack) - 1
                while enter_name != curr_name and i > -1:
                    enter_df_index, enter_timestamp, enter_name = stack[i]
                    i -= 1
    
                if enter_name == curr_name:
                    # remove matched event from the stack
                    del stack[i + 1]
    
                    # Fill in the lists with the matching values if event found
                    matching_events[enter_df_index] = curr_df_index
                    matching_events[curr_df_index] = enter_df_index
    
                    matching_times[enter_df_index] = curr_timestamp
                    matching_times[curr_df_index] = enter_timestamp
                else:
                    continue
    
    df["_matching_event"] = matching_events
    df["_matching_timestamp"] = matching_times
    
    df = df.astype({"_matching_event": "Int32"})

def new_match_events(df):
    matching_events = np.full(len(df), np.nan)
    matching_times = np.full(len(df), np.nan)
    
    # only pairing enter and leave rows
    enter_leave_df = df.loc[
        df["Event Type"].isin(["Enter", "Leave"])
    ]
    
    # list of processes and/or threads to iterate over
    if "Thread" in df.columns:
        exec_locations = set(zip(df["Process"], df["Thread"]))
        has_thread = True
    else:
        exec_locations = set(df["Process"])
        has_thread = False
    
    for curr_loc in exec_locations:
        # only filter by thread if the trace has a thread column
        if has_thread:
            curr_process, curr_thread = curr_loc
            filtered_df = enter_leave_df.loc[
                (enter_leave_df["Process"] == curr_process)
                & (enter_leave_df["Thread"] == curr_thread)
            ]
        else:
            filtered_df = enter_leave_df.loc[
                (enter_leave_df["Process"] == curr_loc)
            ]
    
        stack = []
    
        # Note: The reason that we are creating lists that are
        # copies of the dataframe columns below and iterating over
        # those instead of using pandas iterrows is due to an
        # observed improvement in performance when using lists.
    
        event_types = list(filtered_df["Event Type"])
        df_indices, timestamps, names = (
            list(filtered_df.index),
            list(filtered_df["Timestamp (ns)"]),
            list(filtered_df.Name),
        )
    
        # Iterate through all events of filtered DataFrame
        for i in range(len(filtered_df)):
            curr_df_index, curr_timestamp, evt_type, curr_name = (
                df_indices[i],
                timestamps[i],
                event_types[i],
                names[i],
            )
    
            if evt_type == "Enter":
                # Add current dataframe index and timestamp to stack
                stack.append((curr_df_index, curr_timestamp, curr_name))
            else:
                # we want to iterate through the stack in reverse order
                # until we find the corresponding "Enter" Event
                enter_name, i = None, len(stack) - 1
                while enter_name != curr_name and i > -1:
                    enter_df_index, enter_timestamp, enter_name = stack[i]
                    i -= 1
    
                if enter_name == curr_name:
                    # remove matched event from the stack
                    del stack[i + 1]
    
                    # Fill in the lists with the matching values if event found
                    matching_events[enter_df_index] = curr_df_index
                    matching_events[curr_df_index] = enter_df_index
    
                    matching_times[enter_df_index] = curr_timestamp
                    matching_times[curr_df_index] = enter_timestamp
                else:
                    continue
    
    df["_matching_event"] = matching_events
    df["_matching_timestamp"] = matching_times
    
    df = df.astype({"_matching_event": "Int32"})




In [30]:
%%timeit
original_match_events(trace.events)

93.6 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%%timeit
new_match_events(trace.events)

69.2 ms ± 965 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
dfs = [ v.reset_index() for k, v in df.groupby("Process", observed=True) ]

In [56]:
dfs[0]

,index,Timestamp (ns),Event Type,Name,Thread,Process,Attributes,_matching_event,_matching_timestamp
0,5,4.959588e+04,Instant,ProgramBegin,0,0,"{'attributes': {'Attribute 2': 35455}, 'progra...",<NA>,NaN
1,15,9.251779e+04,Enter,MPI_Init,0,0,{'region': 'Region 36'},39,5.509727e+08
2,39,5.509727e+08,Leave,MPI_Init,0,0,None,15,9.251779e+04
3,53,5.509812e+08,Instant,MeasurementOnOff,0,0,{'measurement_mode': 'MeasurementMode.OFF'},<NA>,NaN
4,124,8.267803e+08,Instant,MeasurementOnOff,0,0,{'measurement_mode': 'MeasurementMode.ON'},<NA>,NaN
...,...,...,...,...,...,...,...,...,...
5971,95538,1.748623e+09,Instant,MpiCollectiveEnd,0,0,"{'collective_op': 'CollectiveOp.ALLREDUCE', 'c...",<NA>,NaN
5972,95563,1.748624e+09,Leave,MPI_Allreduce,0,0,None,95533,1.748600e+09
5973,95583,1.748661e+09,Leave,writeSingleField,0,0,None,95119,1.414827e+09
5974,95614,1.749750e+09,Leave,run,0,0,None,125,8.268101e+08
